In [3]:
%pip install --upgrade pip
%pip install ipywidgets
%pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio
%pip install --upgrade ipywidgets
%pip install notebook
%pip install jupyterlab
%pip install widgetsnbextension

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [5]:
from datasets import load_dataset, DatasetDict
from huggingface_hub import HfFolder
from datasets import DownloadConfig

# Create a download config with your auth token
download_config = DownloadConfig(token=HfFolder.get_token())

common_voice = DatasetDict()

# Load the dataset with the correct auth configuration
common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_11_0",
    "fr",
    split="train+validation",
    trust_remote_code=True,
    download_config=download_config
)

common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_11_0",
    "fr",
    split="test",
    trust_remote_code=True,
    download_config=download_config
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 501123
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 16089
    })
})


In [6]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [7]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="french", task="transcribe")

In [9]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Il est dissous à Trèves.
Decoded w/ special:    <|startoftranscript|><|fr|><|transcribe|><|notimestamps|>Il est dissous à Trèves.<|endoftext|>
Decoded w/out special: Il est dissous à Trèves.
Are equal:             True


In [10]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="french", task="transcribe")

In [11]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/4d487dff50c7da77bd8812384dcadeddf7eece27dd93d909d5c67e4752f45c01/fr_train_0/common_voice_fr_29111041.mp3', 'array': array([1.13686838e-13, 3.41060513e-12, 6.02540240e-12, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 'sampling_rate': 48000}, 'sentence': 'Il est dissous à Trèves.'}


In [12]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [13]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/4d487dff50c7da77bd8812384dcadeddf7eece27dd93d909d5c67e4752f45c01/fr_train_0/common_voice_fr_29111041.mp3', 'array': array([1.45519152e-10, 1.45519152e-10, 8.73114914e-11, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 'sampling_rate': 16000}, 'sentence': 'Il est dissous à Trèves.'}


In [14]:
import torch
from transformers import WhisperFeatureExtractor, WhisperTokenizer
from functools import partial

# Force CUDA device
assert torch.cuda.is_available(), "CUDA is required for this script"
device = torch.device("cuda")

# Initialize the feature extractor and tokenizer directly on GPU
feature_extractor = WhisperFeatureExtractor.from_pretrained(
    "openai/whisper-small",
    device_map="cuda"
)
tokenizer = WhisperTokenizer.from_pretrained(
    "openai/whisper-small", 
    language="french", 
    task="transcribe",
    device_map="cuda"
)

def prepare_dataset(feature_extractor, tokenizer, batch):
    # Process each audio sample in the batch
    input_features_list = []
    for audio_data in batch["audio"]:
        # compute log-Mel input features from input audio array and move to GPU immediately
        features = feature_extractor(
            audio_data["array"], 
            sampling_rate=audio_data["sampling_rate"],
            return_tensors="pt"
        ).input_features[0].cuda()
        input_features_list.append(features)
    
    # Stack features (already on GPU)
    batch["input_features"] = torch.stack(input_features_list)

    # Tokenize all sentences at once with padding
    encoded = tokenizer(
        batch["sentence"], 
        padding=True,
        return_tensors="pt",
        truncation=True,
        max_length=256  # Adjust this value based on your needs
    ).to(device)
    
    batch["labels"] = encoded.input_ids
    batch["attention_mask"] = encoded.attention_mask
    
    return batch

# Create a partial function with the feature_extractor and tokenizer
prepare_dataset_with_deps = partial(prepare_dataset, feature_extractor, tokenizer)

# Now map the dataset using the partial function with GPU processing
with torch.cuda.device(0):  # Ensure operations happen on GPU
    common_voice = common_voice.map(
        prepare_dataset_with_deps, 
        remove_columns=common_voice.column_names["train"], 
        num_proc=1,  # Using single process for GPU
        batched=True,
        batch_size=32  # Adjust based on your GPU memory
    )

Map:   0%|          | 0/501123 [00:00<?, ? examples/s]

Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x791f8a1349a0>:
Traceback (most recent call last):
  File "/root/MultilingualSummarizerAgenticRAGLLM/final/lib/python3.11/site-packages/soundfile.py", line 1295, in vio_read
    data_read = file.readinto(buf)
                ^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


LibsndfileError: Error opening <_io.BufferedReader name='/root/.cache/huggingface/datasets/downloads/extracted/4d487dff50c7da77bd8812384dcadeddf7eece27dd93d909d5c67e4752f45c01/fr_train_0/common_voice_fr_18092124.mp3'>: Format not recognised.

In [2]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.generation_config.language = "french"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-french",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: fr, split: test",
    "language": "fr",
    "model_name": "WhisperSmallFrenchFineTuned",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("humbleakh/WhisperSmallFrenchFineTuned")
processor = WhisperProcessor.from_pretrained("humbleakh/WhisperSmallFrenchFineTuned")


In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="humbleakh/WhisperSmallFrenchFineTuned")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]  
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small French",
    description="Realtime demo for French speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()
